# Domain Adaptation using QLoRA

This notebook demonstrates how to:
1. Extract sentences from a technical PDF
2. Prepare MLM training data
3. Fine-tune a language model using QLoRA

In [1]:
!git clone https://github.com/arminwitte/mistral-peft mistralpeft

fatal: destination path 'mistralpeft' already exists and is not an empty directory.


In [2]:
import os
if not os.getcwd() == "/kaggle/working/mistralpeft":
    os.chdir("/kaggle/working/mistralpeft")
!pwd
!git pull 

/kaggle/working/mistralpeft
Already up to date.


In [3]:
!pip install -r requirements.txt --quiet

In [4]:
from mistralpeft.utils import TextExtractor, MLMPreprocessor, load_base_model, prepare_for_training, generate_response, CLAPreprocessor 
from transformers import Trainer, TrainingArguments, AutoTokenizer
from pathlib import Path

/usr/local/lib/python3.10/dist-packages/nltk/decorators.py:69: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  signature = inspect.formatargspec(regargs, varargs, varkwargs, defaults,


[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [5]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("huggingface")

from huggingface_hub import login
login(secret_value_0) 


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 1. Extract Sentences from PDF

First, we'll extract and clean sentences from the dissertation PDF.

In [6]:
# Initialize extractor
extractor = TextExtractor(language="en")  # or "de" for German

# Process the PDF
pdf_path = "Dissertation.pdf"
extraction_result = extractor.process_document(
    pdf_path,
    output_path="extracted_sentences.json"
)

print(f"Extracted {extraction_result['metadata']['num_sentences']} sentences")

# Preview some sentences
print("\nExample sentences:")
for sentence in extraction_result['sentences'][:3]:
    print(f"- {sentence}")

Reading PDF: 100%|██████████| 315/315 [00:10<00:00, 31.34it/s]


Raw text length: 575529
Cleaned text length: 575529
Sentences: 140
Extracted 140 sentences

Example sentences:
- Technische Universität München Institut für Energietechnik Professur für Thermofluiddynamik Dynamics of Unsteady Heat Transfer and Skin Friction in Pulsating Flow Across a Cylinder Armin Witte Vollständiger Abdruck der von der Fakultät für Maschinenwesen der Technischen Universität München zur Erlangung des akademischen Grades eines DOKTOR – INGENIEURS genehmigten Dissertation. Vorsitzender: Prof. Dr.-Ing. Harald Klein Prüfer der Dissertation: 1. Prof. Wolfgang Polifke, Ph.D. 2. Prof. Dr.-Ing. Jens von Wolfersdorf Die Dissertation wurde am 26.04.2018 bei der Technischen Universität München eingereicht und durch die Fakultät für Maschinenwesen am 09.10.2018 angenommen. Acknowledgments This thesis was conceived at the Thermo-Fluid Dynamics Group of the Technical University of Munich during my time as a research assistant. Financial support was provided by Deutsche Forschungsge

## 2. Prepare MLM Training Data

Now we'll create masked language modeling examples for training.

In [7]:
model_name = "mistralai/Mistral-7B-v0.3"  # Or the specific quantized version if you are using one.
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [8]:
# Initialize preprocessor
preprocessor = MLMPreprocessor(
    tokenizer_name="mistralai/Mistral-7B-v0.3",
    mask_probability=0.15,
    max_length=512,
    seed=42
)

# Create datasets
datasets = preprocessor.process_files(
    ["extracted_sentences.json"],
    output_dir="processed_data",
    train_split=0.9
)

print(f"Created {len(datasets['train'])} training and {len(datasets['val'])} validation examples")

# Preview a training example
example = datasets['train'][0]
print("\nExample input:")
print(preprocessor.tokenizer.decode(example['input_ids']))

Creating MLM examples: 100%|██████████| 140/140 [00:00<00:00, 248.39it/s]


Saving the dataset (0/1 shards):   0%|          | 0/126 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14 [00:00<?, ? examples/s]

Created 126 training and 14 validation examples

Example input:
<s><unk>ische Universität München Institut für<unk>ner<unk>iete<unk>ik Professur für Therm<unk>luiddynamik Dynamics<unk><unk>steady Heat Transfer and Skin Friction in Pulsating<unk> Across<unk> C<unk>inder Armin Witte Vollständiger Abdruck der von der Fakultät für<unk>chinenwesen der Technischen Universität München zur Erlangung des akademischen Grades eines<unk><unk><unk>OR – IN<unk>IEURS genehm<unk> Dissertation. V<unk>itzender: Prof. Dr.-Ing. Harald Klein Prüfer der<unk>sert<unk>: 1.<unk>. Wol<unk>ang<unk>ifke<unk> Ph.D. 2. Prof. Dr.-Ing.<unk>ens von Wolfersdorf<unk> Dissertation wurde<unk> 26<unk>04.201<unk> bei der Technischen Universität München<unk>ereicht und durch die Fakultät für Maschinen<unk>esen am 09.10.<unk>018 angenommen. Acknowledgments This thesis was con<unk> at the Thermo-<unk>uid Dynamics Group<unk> the Technical University of Munich during my time as a research assistant. Financial support<unk> provid

In [9]:
json_file_paths = ["extracted_sentences.json"]
preprocessor = CLAPreprocessor(json_file_paths, tokenizer)
dataset = preprocessor.preprocess()

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


## 3. Load and Prepare Model

We'll now load the base model and prepare it for QLoRA fine-tuning.

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

In [11]:
quantization_config = BitsAndBytesConfig(
        load_in_8bit=True,  # Use load_in_8bit=True for 8-bit quantization
    )

model = AutoModelForCausalLM.from_pretrained(
        model_name, torch_dtype=torch.float16, device_map="auto",
    # quantization_config=quantization_config,
    )

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
from datasets import Dataset
train_test_set = dataset.train_test_split(test_size=0.1)

In [15]:
train_test_set 

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 126
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 14
    })
})

In [13]:
# Load base model and tokenizer
#model, tokenizer = load_base_model()

# Prepare for LoRA training
model = prepare_for_training(
    model,
    lora_r=8,
    lora_alpha=16,
    lora_dropout=0.05
)

## 4. Train the Model

Now we'll fine-tune the model on our domain-specific data.

In [ ]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=3e-4,
    fp16=True,
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    optim="paged_adamw_8bit"
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_test_set['train'],
    eval_dataset=train_test_set['test']
)

# Start training
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
<frozen importlib._bootstrap>:671: ImportWarning: DynamicImporter.exec_module() not found; falling back to load_module()
<frozen importlib._bootstrap>:671: ImportWarning: DynamicImporter.exec_module() not found; falling back to load_module()
/usr/local/lib/python3.10/dist-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_init.py:202: PydanticDeprecatedSince20: The `copy` method is deprecated; use `model_copy` instead. See the docstring of `BaseModel.copy` for details about how to handle `

<IPython.core.display.Javascript object>

## 5. Test the Model

Let's test the fine-tuned model with some domain-specific queries.

In [ ]:
# Example queries about heat transfer and fluid dynamics
queries = [
    "Explain the relationship between pulsating crossflow and heat transfer efficiency.",
    "What are the key factors affecting skin friction in the experimental setup?",
    "Summarize the main findings regarding heat transfer dynamics in the study.",
    "How does the Reynolds number influence the observed phenomena?"
]

# Generate responses
for query in queries:
    print(f"\nQuery: {query}")
    response = generate_response(
        model,
        tokenizer,
        query,
        max_new_tokens=512,
        temperature=0.7
    )
    print(f"Response: {response}")
    print("-" * 80)

## 6. Save the Model

Finally, let's save our fine-tuned model for later use.

In [ ]:
# Save the fine-tuned model
output_dir = Path("./final_model")
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Model saved to {output_dir}")